## Проектирование нормализованной ER-схемы БД

Мы будем следовать Data Vault методологии, которая требует разделения данных на хабы (основные сущности), сателлиты (дополнительные атрибуты сущностей), и линки (связи между сущностями).

### 1.1. Хабы (Hubs)

#### Hub_Questions:
- **Question_ID** (PRIMARY KEY) – уникальный идентификатор вопроса (INTEGER, NOT NULL)
- **Created_At** – дата создания вопроса (DATETIME, NOT NULL)

#### Hub_Answers:
- **Answer_ID** (PRIMARY KEY) – уникальный идентификатор ответа (INTEGER, NOT NULL)
- **Created_At** – дата создания ответа (DATETIME, NOT NULL)

### 1.2. Линки (Links)

### Link_Question_Answers:
- **Link_ID** (PRIMARY KEY) – уникальный идентификатор связи (INTEGER, NOT NULL)
- **Question_ID** (FOREIGN KEY, REFERENCES Hub_Questions) – идентификатор вопроса (INTEGER, NOT NULL)
- **Answer_ID** (FOREIGN KEY, REFERENCES Hub_Answers) – идентификатор ответа (INTEGER, NOT NULL)
- **Is_Accepted** – признак того, что ответ отмечен как правильный (BOOLEAN, NULL)

### 1.3. Сателлиты (Satellites)

#### Sat_Questions_Details:
- **Question_ID** (FOREIGN KEY, REFERENCES Hub_Questions) – идентификатор вопроса (INTEGER, NOT NULL)
- **Title** – заголовок вопроса (TEXT, NOT NULL)
- **Body** – текст вопроса (TEXT, NOT NULL)
- **User_ID** – идентификатор пользователя, который задал вопрос (INTEGER, NOT NULL)
- **Tags** – теги, связанные с вопросом (TEXT, NULL)

#### Sat_Answers_Details:
- **Answer_ID** (FOREIGN KEY, REFERENCES Hub_Answers) – идентификатор ответа (INTEGER, NOT NULL)
- **Body** – текст ответа (TEXT, NOT NULL)
- **User_ID** – идентификатор пользователя, который дал ответ (INTEGER, NOT NULL)
- **Score** – оценка ответа (INTEGER, NULL)

### 1.4. Дополнительные таблицы

#### Hub_Users:
- **User_ID** (PRIMARY KEY) – идентификатор пользователя (INTEGER, NOT NULL)
- **Username** – имя пользователя (TEXT, NOT NULL)
- **Reputation** – репутация пользователя (INTEGER, NOT NULL)

#### Link_User_Questions:
- **Link_ID** (PRIMARY KEY) – уникальный идентификатор связи (INTEGER, NOT NULL)
- **User_ID** (FOREIGN KEY, REFERENCES Hub_Users) – идентификатор пользователя (INTEGER, NOT NULL)
- **Question_ID** (FOREIGN KEY, REFERENCES Hub_Questions) – идентификатор вопроса (INTEGER, NOT NULL)

#### Link_User_Answers:
- **Link_ID** (PRIMARY KEY) – уникальный идентификатор связи (INTEGER, NOT NULL)
- **User_ID** (FOREIGN KEY, REFERENCES Hub_Users) – идентификатор пользователя (INTEGER, NOT NULL)
- **Answer_ID** (FOREIGN KEY, REFERENCES Hub_Answers) – идентификатор ответа (INTEGER, NOT NULL)


In [1]:
# ============================
# БЛОК ИМПОРТОВ
# ============================
# Импорт стандартных библиотек
import sqlite3
from graphviz import Digraph


In [2]:
# Создание подключения к SQLite
conn = sqlite3.connect('stackoverflow.db')
cursor = conn.cursor()

# Создание таблиц для хабов
cursor.execute('''
CREATE TABLE Hub_Questions (
    Question_ID INTEGER PRIMARY KEY,
    Created_At DATETIME NOT NULL
)
''')

cursor.execute('''
CREATE TABLE Hub_Answers (
    Answer_ID INTEGER PRIMARY KEY,
    Created_At DATETIME NOT NULL
)
''')

# Создание таблиц для линков
cursor.execute('''
CREATE TABLE Link_Question_Answers (
    Link_ID INTEGER PRIMARY KEY,
    Question_ID INTEGER NOT NULL,
    Answer_ID INTEGER NOT NULL,
    Is_Accepted BOOLEAN,
    FOREIGN KEY (Question_ID) REFERENCES Hub_Questions(Question_ID),
    FOREIGN KEY (Answer_ID) REFERENCES Hub_Answers(Answer_ID)
)
''')

# Создание таблиц для сателлитов
cursor.execute('''
CREATE TABLE Sat_Questions_Details (
    Question_ID INTEGER NOT NULL,
    Title TEXT NOT NULL,
    Body TEXT NOT NULL,
    User_ID INTEGER NOT NULL,
    Tags TEXT,
    FOREIGN KEY (Question_ID) REFERENCES Hub_Questions(Question_ID)
)
''')

cursor.execute('''
CREATE TABLE Sat_Answers_Details (
    Answer_ID INTEGER NOT NULL,
    Body TEXT NOT NULL,
    User_ID INTEGER NOT NULL,
    Score INTEGER,
    FOREIGN KEY (Answer_ID) REFERENCES Hub_Answers(Answer_ID)
)
''')

cursor.execute('''
CREATE TABLE Hub_Users (
    User_ID INTEGER PRIMARY KEY,
    Username TEXT NOT NULL,
    Reputation INTEGER NOT NULL
)
''')

cursor.execute('''
CREATE TABLE Link_User_Questions (
    Link_ID INTEGER PRIMARY KEY,
    User_ID INTEGER NOT NULL,
    Question_ID INTEGER NOT NULL,
    FOREIGN KEY (User_ID) REFERENCES Hub_Users(User_ID),
    FOREIGN KEY (Question_ID) REFERENCES Hub_Questions(Question_ID)
)
''')

cursor.execute('''
CREATE TABLE Link_User_Answers (
    Link_ID INTEGER PRIMARY KEY,
    User_ID INTEGER NOT NULL,
    Answer_ID INTEGER NOT NULL,
    FOREIGN KEY (User_ID) REFERENCES Hub_Users(User_ID),
    FOREIGN KEY (Answer_ID) REFERENCES Hub_Answers(Answer_ID)
)
''')

# Закрытие подключения
conn.commit()
conn.close()

In [5]:
# Создаем объект графа
er = Digraph('ER Diagram', filename='er_diagram.gv')

# Определяем атрибуты для узлов
er.attr('node', shape='record')

# Хабы (Hubs)
er.node('Hub_Questions', '''{
    Hub_Questions |
    Question_ID : INTEGER (PK)\\l
    Created_At : DATETIME NOT NULL\\l
}''')

er.node('Hub_Answers', '''{
    Hub_Answers |
    Answer_ID : INTEGER (PK)\\l
    Created_At : DATETIME NOT NULL\\l
}''')

er.node('Hub_Users', '''{
    Hub_Users |
    User_ID : INTEGER (PK)\\l
    Username : TEXT NOT NULL\\l
    Reputation : INTEGER NOT NULL\\l
}''')

# Сателлиты (Satellites)
er.node('Sat_Questions_Details', '''{
    Sat_Questions_Details |
    Question_ID : INTEGER (FK)\\l
    Title : TEXT NOT NULL\\l
    Body : TEXT NOT NULL\\l
    User_ID : INTEGER NOT NULL\\l
    Tags : TEXT\\l
}''')

er.node('Sat_Answers_Details', '''{
    Sat_Answers_Details |
    Answer_ID : INTEGER (FK)\\l
    Body : TEXT NOT NULL\\l
    User_ID : INTEGER NOT NULL\\l
    Score : INTEGER\\l
}''')

# Линки (Links)
er.node('Link_Question_Answers', '''{
    Link_Question_Answers |
    Link_ID : INTEGER (PK)\\l
    Question_ID : INTEGER (FK) NOT NULL\\l
    Answer_ID : INTEGER (FK) NOT NULL\\l
    Is_Accepted : BOOLEAN\\l
}''')

er.node('Link_User_Questions', '''{
    Link_User_Questions |
    Link_ID : INTEGER (PK)\\l
    User_ID : INTEGER (FK) NOT NULL\\l
    Question_ID : INTEGER (FK) NOT NULL\\l
}''')

er.node('Link_User_Answers', '''{
    Link_User_Answers |
    Link_ID : INTEGER (PK)\\l
    User_ID : INTEGER (FK) NOT NULL\\l
    Answer_ID : INTEGER (FK) NOT NULL\\l
}''')

# Определяем связи между таблицами
er.edge('Hub_Questions', 'Sat_Questions_Details', label='1-to-1')
er.edge('Hub_Answers', 'Sat_Answers_Details', label='1-to-1')
er.edge('Hub_Questions', 'Link_Question_Answers', label='1-to-many')
er.edge('Hub_Answers', 'Link_Question_Answers', label='1-to-many')
er.edge('Hub_Users', 'Link_User_Questions', label='1-to-many')
er.edge('Hub_Questions', 'Link_User_Questions', label='1-to-many')
er.edge('Hub_Users', 'Link_User_Answers', label='1-to-many')
er.edge('Hub_Answers', 'Link_User_Answers', label='1-to-many')

# Визуализируем граф
er.view()

'er_diagram.gv.pdf'

### Преимущества и недостатки схемы

**Преимущества:**

- **Модульность и гибкость:** Использование Data Vault позволяет добавлять новые атрибуты и связи без необходимости изменения существующей структуры;
- **Историчность данных:** Сателлиты могут хранить исторические изменения атрибутов, что важно для аналитики;
- **Нормализация:** Данные разделены на логические единицы (вопросы, ответы, пользователи), что снижает дублирование данных.

**Недостатки:**

- **Сложность структуры:** Модели Data Vault сложнее в проектировании и поддержке по сравнению с традиционными моделями;
- **Производительность:** За счёт увеличения числа таблиц запросы могут становиться более сложными и требовать больше ресурсов для выполнения.